# Analysing data with Python and MNE

Welcome to the practical session EEG data analysis! This practical session is part of the course Experimentation II at Leiden University. In this session we will study the EEG signals during an Eriksen Flankers Task. We will get acquainted with the typical preprocessing steps, the analysis of the data and the interpretation of the results.

The data that we will be analysing was recorded at Leiden University, with the same equipment that we've used during the data acquisition practical.

The data was acquired during an Eriksen flankers task. This is a conflict task that belongs to the same family of paradigms as the Stroop and Simon task. Participants are instructed to respond as fast as possible to the direction of the central stimulus in an array like this by pressing a response button with the corresponding index finger (in this case the right index finger).

![title](flankers.jpg)

Flanking stimuli around the target are irrelevant, but can prime the correct response (congruent), or the opposite response (incongruent). Immediately after stimulus presentation, the attentional spotlight (the area of stimulus information that is selected for further processing) has not yet been narrowed, so the dominant information derives from the flankers, which may point at the wrong response. As the spotlight is narrowed, the support for the correct response increases. In the ERP you are likely to see initial signs of motor preparation for the hand indicated by the flankers (LRP) and signs of error detection processes (ERN). In addition, the ERP will show large components such as the P300. Prior to the P300, there is a weak negative component that is sensitive to the presence of conflict in the decision processes (N200). We will see to what extent these waves can be identified and quantified. Some of these ERP peaks will be analysed in the following.




### Importing
Now it is time to start our python script. Before we can start loading the data, we need to have the proper tools in place. Obviously we need to import MNE. Next we import matplotlib, for data visualisation, pandas for data analysis, and numpy for working with data structures. If you don't like typing the same words over and over again, you can import libraries as a shorter name, for instance 'pandas' as 'pd'. This saves time later on, but it is optional. Let's ignore the matplotlib.use command.

In [2]:
import mne
import matplotlib
import pandas as pd
import numpy as np

matplotlib.use('TkAgg')

### Creating paths
Next we will setup the folders, so that python knows where to look for what. Type in the path corresponding to the location where you stored the data. To keep things need, we stored the eeg data file and the behavioural data file in two separate foldes.


In [16]:
#Get all the directories
dataPath = '/Users/sebouithol/surfdrive/Leiden/Teaching/Experimentation II/Data Analysis Practical/dataEEG/'
dataPathBehav = '/Users/sebouithol/surfdrive/Leiden/Teaching/Experimentation II/Data Analysis Practical/dataBehavioural/'




### Loading the data
Now we want to create an array with the content of the eeg data file. We do this using the command read_raw_bdf. This command is part of the MNE toolbox, and within MNE of the io (input-output)library, so the full command will read as: nameArray = mne.io.read_raw()

We need to specify the filename within the brackets. For this we also make use of the path that we created above.

In [4]:
filename = "Flanker_EEG_52_12052022.bdf"
fullFileName = (dataPath + filename)

raw_eeg = mne.io.read_raw_bdf(fullFileName, preload=True)

Extracting EDF parameters from /Users/sebouithol/surfdrive/Leiden/Teaching/Experimentation II/Data Analysis Practical/dataEEG/Flanker_EEG_52_12052022.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1778175  =      0.000 ...  3472.998 secs...


### Downsampling
Inspect the output of the code cell above. This will give you information on the data. For instance, you can see that 72 electrodes have been used. That the recording took almost an hour, and that the analogue signal was sampled with a 512Hz frequency. For our purpose this is a bit too high, which will make files unnecessarily big, and computations slow. So we will downsample the data. This will be done using the resample command. We will have to specify the sfreq for this. You can add this in the code below. Use a value of 250.

In [5]:
raw_eeg = raw_eeg.resample(sfreq=250)

Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
2823 events found
Event IDs: [    4     8    16    33    34    35    36 65536 65789]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
2823 events found
Event IDs: [    4     8    16    33    34    35    36 65536 65789]


If you want to check whether it was successful you can use the following command.

In [6]:
raw_eeg.info

<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 72 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 104.0 Hz
 meas_date: 2022-05-12 14:32:28 UTC
 nchan: 73
 projs: []
 sfreq: 250.0 Hz
>

### Rereferencing
You may remember from the lecture that the potential that is recorded at each electrode is not an absolute value, but always a difference between that electrode and another electrode. You may also remember that during the acquisition we have placed two electrodes on the mastoids. We will set the average of these two channels as explicit reference for all the other electrodes.


In [8]:
raw_eeg.set_eeg_reference(ref_channels=['EXG5', 'EXG6'])  # Take average of mastoids as reference


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<RawEDF | Flanker_EEG_52_12052022.bdf, 73 x 868250 (3473.0 s), ~483.6 MB, data loaded>

### Filtering


In [11]:

low_cut = 1
high_cut = 30
filtered_eeg = raw_eeg.copy().filter(low_cut, high_cut, fir_design='firwin')  # Filter between 1Hz and 30Hz


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 sec)



### Loading the behavioural data




In [18]:
fileNameBehav = "flankertask_eyetracker_52_202205121431_format.txt"
fullFileNameBehav = dataPathBehav + fileNameBehav


# Load in behavioral data and make numpy array of trialtype
trials = pd.read_csv(fullFileNameBehav, sep=",")
trials['RT'].replace('None', np.nan, inplace=True)  # Replace "none" answers with "nan" for int processing
trials['RT'] = pd.to_numeric(trials['RT'], errors='coerce', downcast='integer')

trials['condition'] = trials['trialtype'] + ' ' + trials[
    'alert']  # Make condition column, with alert and trialtype
trials_num = trials[['condition']].to_numpy()

# Change coding of trialtype, and reshape to appropriate dimension for later processing
trials_num = np.where(trials_num == "congruent alert", 1, trials_num)
trials_num = np.where(trials_num == "congruent no_alert", 2, trials_num)
trials_num = np.where(trials_num == "incongruent alert", 3, trials_num)
trials_num = np.where(trials_num == "incongruent no_alert", 4, trials_num)
trials_num = np.where(trials_num == "catch alert", 5, trials_num)
trials_num = trials_num.reshape(720)

# Transfer the stimulus EEG triggers to an MNE event file
events = mne.find_events(raw_eeg, uint_cast=True, consecutive=False)

# Delete all markers except for "alert_onset" (34) --> we will use that to orient ourselves
# Note that there were also "alert_onset" triggers in no_alert trials, these were just silent
events_trials = mne.pick_events(events, exclude=[4, 8, 16, 33, 35, 36, 253])

# Merge trials_num and events_trials: replace third column np array with trials_num that contain trial condition
events_trials[:, 2] = trials_num

# alert-centered to flanker centered (+500)
events_trials[:, 0] = events_trials[:, 0] + (500 * (raw_eeg.info['sfreq'] / 1000))
# Dictionary of numbers
event_dict = {"congruent alert": 1, "congruent no_alert": 2,
              "incongruent alert": 3, "incongruent no_alert": 4,
              "catch alert": 5}

2813 events found
Event IDs: [  4   8  16  33  34  35  36 253]


In [21]:
stimulus_epoch = (-0.6, 0.7)
tmin, tmax = stimulus_epoch  # epoch from 0.6s before flanker onset to 0.7s after it

epochs = mne.Epochs(filtered_eeg, events_trials, event_dict, tmin, tmax, baseline=None,
                    metadata=trials, preload=True)

Adding metadata with 11 columns
720 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 720 events and 326 original time points ...
0 bad epochs dropped


### Artifact rejection
You may remember from the data acquisition practical that when the participant blinks, this will create big artifacts. So we want our software to detect eye blinks in the EOG channels, and correct the signal during those events. An independent component analysis (ICA) can do this for you.



In [ ]:
# ICA FOR EOG ARTIFACT DETECTION ----------------------------------------------------------------------------
# Set up ICA
from mne import epochs
from mne.preprocessing import ICA

method = 'picard'

n_components = 40 # select n_components by explained variance of PCA
random_state = 23 # Random seed

ica = ICA(n_components=n_components, method=method, random_state=random_state)
picks = mne.pick_types(epochs.info, eeg=True, eog=False,
                       stim=False)

# Fit ICA on non-rejected epochs
ica.fit(epochs[~reject_log.bad_epochs], picks=picks, reject=None)

ica.exclude = []

# Then, find which ICs match the EOG pattern and put them in ica.exclude
eog_indices, eog_scores = ica.find_bads_eog(epochs)
ica.exclude = eog_indices

# plot ICs applied to raw data, with ica.exclude highlighted. Check and see if I want to remove/add ICA's
ica_components_plot = ica.plot_components(title="ica components of rejected epochs")
ica_plot_sources = ica.plot_sources(epochs[~reject_log.bad_epochs], block=True)

# Finally, save and apply the ICA exclude to the data
ica.save(file_loc + f'EEG/ica_eog_solution_{locked}_{subj}-ica.fif', overwrite=True)
ica.apply(epochs, exclude=ica.exclude)

epochs.info['bads'] = [] # Make bads list empty again, we'll let autoreject handle this
epochs.save(file_loc + f'EEG/ica_cleaned_epochs_{locked}_{subj}-epo.fif', overwrite=True)
print(f"Saved ica-cleaned {locked} epochs")



### Filtering



In [ ]:
low_cut = 1
high_cut = 30
filtered_eeg = raw_eeg.copy().filter(low_cut, high_cut, fir_design='firwin')  # Filter between 0.1Hz and 40Hz


We will have to indicate which channels will be used as reference. Let's use the average of the two mastoids for this

In [4]:
raw.set_eeg_reference(ref_channels=['EXG5', 'EXG6']) # Take average of mastoids as reference

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<RawEDF | Eriksen001.bdf, 21 x 584960 (2285.0 s), ~93.7 MB, data loaded>

In [29]:
biosemi_montage = mne.channels.make_standard_montage("biosemi16")
raw.set_montage(biosemi_montage, on_missing='ignore')
raw.set_channel_types({'EXG1': 'eog', 'EXG2': 'eog', 'EXG3': 'eog', 'EXG4': 'eog', 'EXG5': 'eog', 'EXG6': 'eog'})


Now let's plot the raw data for a quick visual inspection

In [10]:
raw.plot()

You will see that after an initial period, EXG5 and EXG6 are almost a flat line, why is this the case? Why is it not a complete flat line?

In [ ]:
print(raw.info)

Probably depending on the screen, but it could be that the channel names  list has been truncated. When we are specifically interested in the channels we can use the ch_names command.

In [11]:
print(raw.ch_names)

In [110]:
montage=mne.channels.make_standard_montage('standard_1020')

In [111]:
#numChan = 21
#for channel in range (0,numChan):

#mne.channels.get_builtin_montages()
#montage=mne.channels.make_standard_montage(kind="biosemi32")
raw.set_montage(montage); #on_missing='ignore'
#print(raw.ch_names)

ValueError: DigMontage is only a subset of info. There are 14 channel positions not present in the DigMontage. The required channels are:

['A10', 'A12', 'A13', 'A14', 'A18', 'A31', 'B6', 'B13', 'B15', 'B16', 'B17', 'B18', 'B19', 'B23'].

Consider using inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.

In [ ]:
print(raw.ch_names)

In [7]:
raw.plot_sensors(kind='3d', ch_type='eeg')

RuntimeError: No valid channel positions found

In [ ]:
raweeg.compute_psd(fmax=50).plot